In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [26]:
r = requests.get(URL_TEMPLATE)
print(r.status_code)

200


In [27]:
soup = bs(r.text, "html.parser")
parsed_data = soup.find_all('a', class_='am')

In [28]:
parsed_data[:2]

[<a class="am" data="ayU3QmwlQUFrJUE5JUNBeSU5RnB3biVBRWclQUUlODclODAlOURvdWwlQUY=|8C9y2yNHk" href="/msg/lv/real-estate/flats/riga/agenskalns/egdcd.html" id="dm_52601044">Īpašnieks pārdod plašu 2-istabu dzīvokli Botāniskā dārza rajonā.</a>,
 <a class="am" data="JThBaSU4NSU4MiVBOCU3QiU5RCVFQSU4QWolODglODUlQTR3JTk5JUExJTkwaCU4RCU4NSVBOHYlOUMlOUUlODlk|Y4TPsFhn" href="/msg/lv/real-estate/flats/riga/vecmilgravis/bxlkol.html" id="dm_53174001"><b>Pilnībā aprīkots 3 istabu dzīvoklis ar lodžiju klusā, zaļā apkai</b></a>]

In [29]:
parsed_data[0].get('href')

'/msg/lv/real-estate/flats/riga/agenskalns/egdcd.html'

In [30]:
def get_link(link, time_sleep, page_num):
    
    time.sleep(time_sleep)
    link = link + 'page' + str(page_num) + '.html'
    r = requests.get(link)
    
    if r.status_code!=200:
        return 
    
    soup = bs(r.text, "html.parser")
    parsed_data = soup.find_all('a', class_='am')
    
    pars_links = []
    
    for data in parsed_data:
        pars_links.append(data.get('href'))
        
    return pars_links

In [52]:
link_array = []
URL_TEMPLATE = "https://www.ss.lv/lv/real-estate/flats/riga/all/sell/"

In [62]:
for i in tqdm(range(1,114)):
    link_array = link_array + get_link(URL_TEMPLATE, 1, i)

  0%|          | 0/113 [00:00<?, ?it/s]

In [54]:
link_array[:10]

['/msg/lv/real-estate/flats/riga/centre/aepxp.html',
 '/msg/lv/real-estate/flats/riga/agenskalns/egdcd.html',
 '/msg/lv/real-estate/flats/riga/vecmilgravis/bxlkol.html',
 '/msg/lv/real-estate/flats/riga/ilguciems/cnxbj.html',
 '/msg/lv/real-estate/flats/riga/imanta/ajkmm.html',
 '/msg/lv/real-estate/flats/riga/mezhapark/becpke.html',
 '/msg/lv/real-estate/flats/riga/ziepniekkalns/akmdm.html',
 '/msg/lv/real-estate/flats/riga/bolderaya/cioko.html',
 '/msg/lv/real-estate/flats/riga/teika/eicjk.html',
 '/msg/lv/real-estate/flats/riga/teika/edgme.html']

In [64]:
# обратите внимание, от нашего URL остался только домен, остальное мы будет добавлять из сохраненных ссылок
URL_TEMPLATE = "https://www.ss.lv"

In [63]:
len(link_array)

9720

In [65]:
URL_TEMPLATE += link_array[0]

In [66]:
r = requests.get(URL_TEMPLATE)
print('Error status', r.status_code)

Error status 200


In [67]:
soup = bs(r.text, "html.parser")
# parsed_data = soup.find_all('td', class_='msga2-o pp6')

In [68]:
parsed_data = soup.find_all('td', class_='ads_opt')
parsed_data

[<td class="ads_opt" id="tdo_20" nowrap=""><b>Rīga</b></td>,
 <td class="ads_opt" id="tdo_856" nowrap=""><b>centrs</b></td>,
 <td class="ads_opt" id="tdo_11" nowrap=""><b>Matīsa 41</b> <span class="td15">[<a class="ads_opt_link_map" href="javascript:;" id="mnu_map" onclick="mnu('map',1,1,'/lv/gmap/fTgTeF4QAzt4FD4eFFM=.html?mode=1&amp;c=56.9558095, 24.1363359, 14');return false;">Karte</a>]</span></td>,
 <td class="ads_opt" id="tdo_1" nowrap="">4</td>,
 <td class="ads_opt" id="tdo_3" nowrap="">100 m²</td>,
 <td class="ads_opt" id="tdo_4" nowrap="">4/6</td>,
 <td class="ads_opt" id="tdo_6" nowrap="">P. kara</td>,
 <td class="ads_opt" id="tdo_2" nowrap="">Mūra</td>]

In [69]:
# все данные получили
for i in parsed_data:
    print(i.get_text("|"))

Rīga
centrs
Matīsa 41| |[|Karte|]
4
100 m²
4/6
P. kara
Mūra


In [70]:
# теперь попробуем достать карту 
parsed_map = soup.find_all('a', class_='ads_opt_link_map')
print(parsed_map[0]['onclick'])

mnu('map',1,1,'/lv/gmap/fTgTeF4QAzt4FD4eFFM=.html?mode=1&c=56.9558095, 24.1363359, 14');return false;


In [71]:
# и тогда достанем и описание апартаментов 
parsed_text = soup.find_all('div', id='msg_div_msg')
parsed_text[0].get_text(" | ")

# обратите внимание, что в контейнере id=msg_div_msg захватывает и данные по атрибутам которые мы уже взяли, 
# это связано с принципом вложенности, в данном контейнере и находяться наши теги td

'\n | \r\n\r\nPārdodu plašu un gaišu dzīvokli. Pārdomāts un ērts plānojums: | \r\n - viesistaba (apvienota ar virtuves zonu),  | \r\n- divas izolētas guļamistabas,  | \r\n- izolēts kabinets (vai trešā guļamistaba),  | \r\n- plašs gaitenis,  | \r\n- plaša un ērta vannas istaba ar vannu, papildus vēl atsevišķa wc,  | \r\n- neliela noliktavas telpa.  | \r\nVirtuvē visa nepieciešamā sadzīves tehnika (keramiskā plīts, cepeškrāsns, ledusskapis, mikroviļņu krāsns, trauku mazgājamā mašīna, tvaiku nosūcējs).  | \r\nDzīvoklis ar lielisku skaņas izolāciju, ļoti silts, augstie griesti. Dzīvokļa logi vērsti gan uz ielas pusi, gan iekšpagalmu. Kāpņu telpa ar durvju kodu. Bez lifta.  | \r\nDzīvoklim ir pilsētas komunikācijas - ūdens, kanalizācija un centrālā apkure. Dzīvoklī mainīta elektroinstalācija, ūdens caurules, radiatori.  | \r\nĒrta infrastruktūra un sabiedriskā transporta kustība. Tuvumā Bērnu pasaule, Ziedoņdārzs, veikali, aptiekas, skolas.  | \r\nIr iespēja pagalmā novietot automašīnu. Aps

In [72]:
# и как же без цены
parsed_price = soup.find_all('td', class_='ads_price')
parsed_price[0].get_text()

'190 000 € (1 900 €/m²)'

In [73]:
# по традиции что выполняется больше 1го раза мы пишем в функцию. 
# напишем функцию в которую мы будем передавать ссылку. а все данные в ввиде массива будут приходить ответом 

In [74]:
def get_data_link(url, time_sleep):
    
    page_array = []
    time.sleep(time_sleep)
    
    # добавляем к существующему домену
    link = "https://www.ss.lv"
    link += url
    
    r = requests.get(link)
    if r.status_code!=200:
        return 
    
    soup = bs(r.text, "html.parser")
        
    # данные 
    parsed_data = soup.find_all('td', class_='ads_opt')   
    # координаты
    parsed_map = soup.find_all('a', class_='ads_opt_link_map')   
        
    # цена
    parsed_price = soup.find_all('td', class_='ads_price')    
    # описание 
    parsed_text = soup.find_all('div', id='msg_div_msg')
    
    
    for data in parsed_data:
        page_array.append(data.get_text("|"))

    if len(parsed_map)==1:
        page_array.append(parsed_map[0]['onclick'])
    else:
        page_array.append('')
    
    page_array.append(parsed_price[0].get_text())       
    page_array.append(parsed_text[0].get_text(" | "))
    
    return page_array

In [49]:
#data_array = []

#data_array.append(get_data_link(link_array[447], 1))

In [75]:
data_array

[['Rīga',
  'centrs',
  'Čaka 134| |[|Karte|]',
  '2',
  '76 m²',
  '4/6/lifts',
  'Jaun.',
  'Mūra',
  'Lodžija, Parkošanas vieta',
  "mnu('map',1,1,'/lv/gmap/fTgTeF4QAzt4FD4eFFM=.html?mode=1&c=56.9617433, 24.1469984, 14');return false;",
  '170 000 € (2 236.84 €/m²)',
  '\n | \r\n\r\nPiedāvājam iegādāties plašu 2 istabu dzīvokli ar āra terasi, 2018. gadā būvētā jaunā projekta ēkā \'"Aizkulises". Ēka ar interesantu, mūsdienīgu arhitektūru, atrodas tālajā centrā, netālu no Grīziņkalna. Dzīvoklis ar pilnu apdari un kvalitatīvu remontu ir izvietots ēkas 4 stāvā. Tas tiek pārdots ar visām mēbelēm, iekārtām un sadzīves tehniku. Pēc plānojuma - plašs gaitenis ar garderobes skapi, vannas istaba ar visu nepieciešamo, viesistaba ar virtuves zonu, guļamistaba ar plašu garderobes telpu. Viesistabu no virtuves zonas iespējams atdalīt ar bīdāmu stikla sienu. Kopējā dzīvokļa platība ir 75.7 m2, tajā skaitā ārā terase 11.7 m2. Zemi komunālie maksājumi, siltuma skaitītāji katram dzīvoklim, apkures ra

In [76]:
# запишем все данные 
data_array = []

for links in tqdm(link_array):
    # у нас будет 1470 запросов. довольно много, иногда лучше увеличить время ожидания, чем поймать бан 
    # в нашем случае парсинг + выгрузка (с тайм аутом) занимают ~1,8 сек, этого обычно достаточно достаточно
    # из опыта могу сказать что в среднем устанавливают до 500 итераций в минуту или 8.3 в секунду
    data_array.append(get_data_link(links, 0.25))

  0%|          | 0/9720 [00:00<?, ?it/s]

In [77]:
len(data_array)

9720

In [79]:
# в части данных присутсвует дополнительное поле "удобства", добавим строку для формирования df 
# в части данных присутсвует дополнительное поле "Kadastra numurs", добавим строку для формирования df 
data_array_upd = []
for i in data_array:
    if len(i)==11:
        i.insert(8, '')
    if len(i)==12:
        i.insert(8, '')
    data_array_upd.append(i)
    
data_array_upd_1 = []
for i in data_array_upd:
    if len(i)==12:
        i.insert(8, '')
    data_array_upd_1.append(i)

In [80]:
data_array_upd=data_array_upd_1

In [81]:
df = pd.DataFrame(data_array_upd, columns=['city', 'district','street','rooms','area','floor','seria','house_type','kadastr_numb','facilities', 'map','price','all_data'])

In [82]:
df.sample(2)

,city,district,street,rooms,area,floor,seria,house_type,kadastr_numb,facilities,map,price,all_data
8709,Rīga,Iļģuciems,Baltā 11 k-1| |[|Karte|],2,48 m²,1/3,Hrušč.,Ķieģeļu,,Parkošanas vieta,"mnu('map',1,1,'/lv/gmap/fTgTeF4QAzt4FD4eFFM=.h...",38 350 € (798.96 €/m²),\n | \r\n\r\nPārdošanā nonācis divu istabu dzī...
7284,Rīga,Āgenskalns,Mērsraga 9a| |[|Karte|],2,59 m²,2/3,Renov.,Mūra,,,"mnu('map',1,1,'/lv/gmap/fTgTeF4QAzt4FD4eFFM=.h...",92 108 € (1 561.15 €/m²),\n | \r\n\r\nĪpašnieks pārdod 2-istabu dzīvokl...


In [ ]:
# преступим к очистке данных

In [83]:
df['city'].unique()

array(['Rīga'], dtype=object)

In [84]:
df['district'].unique()

array(['centrs', 'Āgenskalns', 'Vecmīlgrāvis', 'Iļģuciems', 'Imanta',
       'Mežaparks', 'Ziepniekkalns', 'Bolderāja', 'Teika', 'Mangaļi',
       'Ķengarags', 'Krasta r-ns', 'Maskavas priekšpilsēta', 'Purvciems',
       'Aplokciems', 'Vecrīga', 'Sarkandaugava', 'Pļavnieki', 'Dārzciems',
       'Daugavgrīva', 'Dzegužkalns', 'Torņakalns', 'Mežciems', 'Jugla',
       'Šampēteris-Pleskodāle', 'Zolitūde', 'Jaunciems', 'Ķīpsala',
       'Bieriņi', 'Katlakalns', 'Čiekurkalns', 'Grīziņkalns', 'Bukulti',
       'Berģi', 'Klīversala', 'Cits', 'Voleri', 'Rumbula', 'Šķirotava',
       'VEF', 'Dreiliņi', 'Mangaļsala', 'Brekši', 'Vecāķi', 'Dārziņi',
       'Zasulauks'], dtype=object)

In [85]:
df[['data_street', 'map_link']] = df['street'].str.split(pat='|', n=1 , expand=True )

In [86]:
df = df.drop(['city','map_link','street','kadastr_numb'], axis=1)

In [87]:
df[['cur_floor', 'max_floor']] = df['floor'].str.split(pat='/', n=1 , expand=True )

In [88]:
df = df.drop(['floor'], axis=1)

In [93]:
df.to_excel('df.xlsx', index=False)

In [96]:
df['map']=df['map'].fillna('-1')

In [97]:
def get_cord(row):
    # ищем стартовую точку 
    point_start = row['map'].find('c=') + 2
    
    first_coma = row['map'][point_start:].find(',') + 1

    second_coma = row['map'][point_start+first_coma:].find(',')
    
    cord = row['map'][point_start:point_start+first_coma+second_coma]
    
    return cord    

In [98]:
df['cord_map'] = df.apply(get_cord, axis=1)

In [99]:
df[['len', 'lon']] = df['cord_map'].str.split(pat=',', n=1 , expand=True )
df = df.drop(['cord_map'], axis=1)
df = df.drop(['map'], axis=1)

In [100]:
df['area'] = df['area'].apply(lambda x: x.replace(' m²',''))

In [101]:
df[['price_eur', 'else_price']] = df['price'].str.split(pat='(', n=1 , expand=True )

In [102]:
df = df.drop(['price','else_price'], axis=1)

In [103]:
df['max_floor'].unique()

array(['6', '2', '9/lifts', '5', '3', '4', '7/lifts', '14/lifts', '7',
       '6/lifts', '11/lifts', '16/lifts', '5/lifts', '10/lifts',
       '13/lifts', '1', '18/lifts', '9', '8/lifts', '12/lifts', '12',
       '23/lifts', '24/lifts', '4/lifts', '16', '21/lifts', '22/lifts',
       '2/lifts', '10', '25/lifts', '23', '13', '8', '15/lifts', '22',
       '1/lifts', '24', '3/lifts', '18', '30/lifts', '11', '19/lifts',
       '31/lifts', '14'], dtype=object)

In [104]:
df[['total_floor', 'lift']] = df['max_floor'].str.split(pat='/', n=1 , expand=True )
df = df.drop(['max_floor'], axis=1)

In [107]:
df[['price', 'currency']] = df['price_eur'].str.split(pat=' €', n=1 , expand=True )

In [109]:
df = df.drop(['price_eur'], axis=1)

In [115]:
df['price']=df['price'].fillna('-1')

In [116]:
df['price'] = df['price'].apply(lambda x: x.replace(' ',''))

In [118]:
df['lon'] = df['lon'].fillna('-1')
df['len'] = df['len'].fillna('-1')

df.loc[df['len']=='', 'len'] = '-1'
df.loc[df['lon']=='', 'lon'] = '-1'

In [119]:
df['len'] = df['len'].apply(lambda x: x.replace(' ',''))
df['lon'] = df['lon'].apply(lambda x: x.replace(' ',''))

In [120]:
df = df[['district','data_street','rooms','area','price','cur_floor','total_floor', 'lift', 'seria','house_type','facilities','len','lon','all_data']]

In [124]:
df=df[df['rooms']!='Citi']

In [126]:
df['cur_floor']=df['cur_floor'].replace('1.00', 1)

In [127]:
df['rooms'] = df['rooms'].astype('int64')
df['area'] = df['area'].astype('float64')
df['price'] = df['price'].astype('int64')
df['cur_floor'] = df['cur_floor'].astype('int64')
df['total_floor'] = df['total_floor'].astype('int64')
df['len'] = df['len'].astype('float64')
df['lon'] = df['lon'].astype('float64')

In [128]:
df_all_data = df['all_data']

In [129]:
df = df.drop(['all_data'], axis=1)

In [130]:
df['facilities'].unique()

array(['', 'Parkošanas vieta', 'Lodžija', 'Lodžija, Parkošanas vieta',
       'Balkons', 'Balkons, Parkošanas vieta', 'Terase, Parkošanas vieta',
       'Balkons, Lodžija, Parkošanas vieta', 'Balkons, Terase',
       'Balkons, Terase, Parkošanas vieta', 'Balkons, Lodžija',
       'Balkons, Lodžija, Terase', '01009162413', '01009206586',
       'Balkons, Lodžija, Terase, Parkošanas vieta', 'Terase',
       'Lodžija, Terase, Parkošanas vieta', '01009227150', 'Pirts',
       '01009253495', '01009182346', '02009133230', '01009262288',
       'Balkons, Pirts, Parkošanas vieta', '01000750010', 'Terase, Pirts',
       '01009264745', '01009278620',
       'Lodžija, Terase, Pirts, Parkošanas vieta', '01009102986',
       '01009243991', '0100 904 5609', '01009279671', '01009155522',
       '01009079976', 'Pirts, Parkošanas vieta', '01000120127',
       '01000370040', '01009068028', 'Lodžija, Pirts, Parkošanas vieta',
       '0100 906 8724', '01000200016001', '01000890172002062',
       'Balkons,

In [131]:
arr_facilities = ['Terase', 'Terase, Parkošanas vieta', 'Lodžija',
       'Lodžija, Parkošanas vieta', 'Parkošanas vieta', 'Balkons',
       'Balkons, Lodžija, Terase',
       'Pirts, Parkošanas vieta', 'Balkons, Parkošanas vieta',
       'Balkons, Lodžija', 'Balkons, Lodžija, Parkošanas vieta',
       'Balkons, Lodžija, Terase, Parkošanas vieta',
       'Balkons, Terase, Parkošanas vieta',
       'Lodžija, Terase, Parkošanas vieta', 'Lodžija, Terase',
       'Terase, Pirts']

In [132]:
df['lift'] = np.where(df['lift']=='lifts',1,0)

In [133]:
df['facilities'] = np.where(df['facilities'].isin(arr_facilities),df['facilities'],'')

In [134]:
df.head()

,district,data_street,rooms,area,price,cur_floor,total_floor,lift,seria,house_type,facilities,len,lon
0,centrs,Matīsa 41,4,100.0,190000,4,6,0,P. kara,Mūra,,56.955810,24.136336
1,Āgenskalns,Baldones 10,2,56.0,36400,1,2,0,P. kara,Koka,Parkošanas vieta,56.948678,24.063496
2,Vecmīlgrāvis,Ledus 2,3,63.0,53000,1,9,1,602.,Paneļu,Lodžija,57.037709,24.095287
3,Iļģuciems,Lilijas 14,2,58.0,31500,2,2,0,P. kara,Koka,,56.965051,24.064370
4,Imanta,Zentenes 18,2,50.0,53900,9,9,1,602.,Paneļu,,56.961812,24.003519


In [135]:
len(df)

9693

In [136]:
df=df.drop_duplicates()

In [137]:
len(df)

3357

In [138]:
df.head()

,district,data_street,rooms,area,price,cur_floor,total_floor,lift,seria,house_type,facilities,len,lon
0,centrs,Matīsa 41,4,100.0,190000,4,6,0,P. kara,Mūra,,56.955810,24.136336
1,Āgenskalns,Baldones 10,2,56.0,36400,1,2,0,P. kara,Koka,Parkošanas vieta,56.948678,24.063496
2,Vecmīlgrāvis,Ledus 2,3,63.0,53000,1,9,1,602.,Paneļu,Lodžija,57.037709,24.095287
3,Iļģuciems,Lilijas 14,2,58.0,31500,2,2,0,P. kara,Koka,,56.965051,24.064370
4,Imanta,Zentenes 18,2,50.0,53900,9,9,1,602.,Paneļu,,56.961812,24.003519


In [140]:
df.to_csv('df_for_sale_riga_11102023.csv',index=False)